Setup Environment

In [58]:
# from msp.dataset import download_dataset, load_dataset, combine_dataset, update_dataset
from msp.composition import generate_random_compositions, sample_random_composition
from msp.forcefield import MDL_FF, MACE_FF, M3GNet_FF
from msp.optimizer.globalopt.basin_hopping import BasinHoppingASE, BasinHoppingBatch
from msp.utils.objectives import UpperConfidenceBound, Energy
from msp.structure.structure_util import dict_to_atoms, init_structure, atoms_to_dict
from msp.validate import read_dft_config, setup_DFT, Validate
import pickle as pkl
import json
import numpy as np
import ase
import torch
from ase import io
from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.io.ase import AseAtomsAdaptor
from matminer.featurizers.site import CrystalNNFingerprint
from matminer.featurizers.structure import SiteStatsFingerprint

Download dataset, specify configurations, and intiailize forcefield (we use MDL_FF but it can be a force field from another library).

Note: Mace may have a memory leak.

In [59]:
my_dataset = json.load(open("../data/data_subset_msp.json", "r"))
train_config = 'mdl_config.yml'     # Configuration for mdl model
forcefield = MDL_FF(train_config, my_dataset)   # Forcefield
forcefield_mace = MACE_FF()
forcefield_m3gnet = M3GNet_FF()

INFO:root:Using PyTorch automatic mixed-precision
INFO:root:GPU is available: True, Quantity: None
INFO:root:Dataset(s) used:
INFO:root:Dataset length: ('train', 22209)
INFO:root:Dataset length: ('val', 1233)
INFO:root:Dataset length: ('test', 1233)
INFO:/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/matgl/utils/io.py:Using cached local file at /global/homes/r/rithwiks/.cache/matgl/M3GNet-MP-2021.2.8-PES/model.pt...
INFO:/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/matgl/utils/io.py:Using cached local file at /global/homes/r/rithwiks/.cache/matgl/M3GNet-MP-2021.2.8-PES/state.pt...
INFO:/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/matgl/utils/io.py:Using cached local file at /global/homes/r/rithwiks/.cache/matgl/M3GNet-MP-2021.2.8-PES/model.json...


Attempting to load checkpoint...
model loaded successfully
loaded from ../pretrained_models/best_checkpoint.pt
Recent checkpoint loaded successfully.


/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/matgl/apps/pes.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/matgl/apps/pes.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/matgl/apps/pes.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().deta

Initialize a Structure Predictor. We currently use basin hopping to optimize structures given compositions

Note: MSP offers 2 types of Basin Hopping optimizers. 
- BasinHoppingBatch
    - Uses a forcefield and pytorch optimizers
    - Can optimize in batches
    - Can minimize any objective function in MSP
- BasinHoppingASE 
    - Uses an ASE calculator that can use a forcefield
    - Unable to optimize in batches
    - Only minimizes energy

Possible perturbs include pos, cell, atomic_num, add, remove, swap

In [60]:
predictor_ase = BasinHoppingASE(forcefield, hops=5, steps=100, optimizer="FIRE", dr=0.5, perturbs=['pos', 'cell'])

predictor_batch = BasinHoppingBatch(forcefield, hops=5, steps=100, dr=0.6, optimizer='Adam', batch_size=32, perturbs=['pos', 'cell'])

predictor_mace = BasinHoppingASE(forcefield_mace, hops=5, steps=100, optimizer="FIRE", dr=0.5, max_atom_num=84, perturbs=['pos', 'cell'])

predictor_m3gnet = BasinHoppingASE(forcefield_m3gnet, hops=5, steps=100, optimizer="FIRE", dr=0.5, max_atom_num=84, perturbs=['pos', 'cell'])

INFO:root:MDLCalculator instantiated from a dictionary.
INFO:root:MDLCalculator: setting up torchmd_etEarly for calculation
INFO:root:MDLCalculator: weights for model No.1 loaded from ../pretrained_models/best_checkpoint.pt
INFO:root:CUDA version: 12.1, CUDA device: 0


Using Materials Project MACE for MACECalculator with /global/homes/r/rithwiks/.cache/mace/5f5yavf3
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


Optionally, you can train the forcefield if you do not have a pretrained checkpoint or update the forcefield using the dataset if you did load a checkpoint.

In [61]:
# forcefield.train(my_dataset, .09, .05, .05, max_epochs=1)
# forcefield.update(my_dataset, .09, .05, .05, max_epochs=1)

Generate/initialize compositions (list of atomic numbers)
3 options:
- generate_random_compositions
- sample_random_composition
- Manually specify composition lists

In [62]:
# compositions = sample_random_composition(dataset=my_dataset, n=5)
# compositions = generate_random_compositions(dataset=my_dataset, n=5, max_elements=5, max_atoms=10)
compositions = [[22, 22, 8, 8, 8, 8] for _ in range(5)]

Initialize structures for each composition
- Random intialization (pyxtal = False)
- Pyxtal from_random (pyxtal = True)
    - Performs better for small number of hops (1-20)
- Initialize from cif file

In [63]:
initial_structures = [init_structure(c, pyxtal=False) for c in compositions]
read_structure = ase.io.read("init.cif")
# initial_structures=[atoms_to_dict([read_structure], loss=[None])]

Batch Optimization using Objective Functions

In [64]:
# objective_func = UpperConfidenceBound(c=0.1)
objective_func = Energy()
total_list_batch, minima_list_batch = predictor_batch.predict(initial_structures, objective_func, batch_size=32)

# Optionally save cif files of best structures
# minima_list_batch = dict_to_atoms(minima_list_batch)
# for j, minima in enumerate(minima_list_batch):
#     filename = "iteration_"+str(i)+"_structure_"+str(j)+"_mdl_batch.cif"
#     ase.io.write(filename, minima)

device: cuda
HOP 0 took 2.647949457168579 seconds
	Structure 0
		HOP 0 previous energy [48.789543]
		HOP 0 optimized energy [-49.170334]
	Structure 1
		HOP 0 previous energy [43.19364]
		HOP 0 optimized energy [-48.60825]
	Structure 2
		HOP 0 previous energy [11.796659]
		HOP 0 optimized energy [-42.88911]
	Structure 3
		HOP 0 previous energy [28.462786]
		HOP 0 optimized energy [-42.75117]
	Structure 4
		HOP 0 previous energy [52.523464]
		HOP 0 optimized energy [-52.23693]
HOP 0 took 2.647949457168579 seconds
device: cuda
HOP 1 took 2.826434373855591 seconds
	Structure 0
		HOP 1 previous energy [-39.956905]
		HOP 1 optimized energy [-44.58764]
	Structure 1
		HOP 1 previous energy [-42.945076]
		HOP 1 optimized energy [-44.73047]
	Structure 2
		HOP 1 previous energy [-12.835407]
		HOP 1 optimized energy [-50.119267]
	Structure 3
		HOP 1 previous energy [-12.517702]
		HOP 1 optimized energy [-44.398594]
	Structure 4
		HOP 1 previous energy [-44.434418]
		HOP 1 optimized energy [-53.537

ASE Calculator Optimization using forcefield

In [65]:
total_list_ase, minima_list_ase = predictor_ase.predict(initial_structures)
total_list_mace, minima_list_mace = predictor_mace.predict(initial_structures)    
total_list_m3gnet, minima_list_m3gnet = predictor_m3gnet.predict(initial_structures)   

Structure 0
	HOP 0 took 3.5940489768981934 seconds
	HOP 0 previous energy 48.78953170776367
	HOP 0 optimized energy -53.955596923828125
	HOP 1 took 3.436398983001709 seconds
	HOP 1 previous energy -23.65780258178711
	HOP 1 optimized energy -60.20496368408203
	HOP 2 took 3.5316805839538574 seconds
	HOP 2 previous energy -60.20496368408203
	HOP 2 optimized energy -60.3326416015625
	HOP 3 took 3.6263351440429688 seconds
	HOP 3 previous energy -45.705352783203125
	HOP 3 optimized energy -58.79951095581055
	HOP 4 took 3.8318207263946533 seconds
	HOP 4 previous energy -43.999290466308594
	HOP 4 optimized energy -54.17325210571289
Structure 0 Min energy -60.3326416015625
Structure 1
	HOP 0 took 3.6196887493133545 seconds
	HOP 0 previous energy 43.19363784790039
	HOP 0 optimized energy -54.31074142456055
	HOP 1 took 3.7054824829101562 seconds
	HOP 1 previous energy -7.811008930206299
	HOP 1 optimized energy -45.12678527832031
	HOP 2 took 3.439361572265625 seconds
	HOP 2 previous energy 55.9634

/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/matgl/layers/_basis.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  root = torch.tensor(roots[i])


	HOP 0 took 5.674908638000488 seconds
	HOP 0 previous energy 27.733829498291016
	HOP 0 optimized energy -28.20461082458496
	HOP 1 took 6.284326076507568 seconds
	HOP 1 previous energy -28.844411849975586
	HOP 1 optimized energy -39.29130172729492
	HOP 2 took 6.001939296722412 seconds
	HOP 2 previous energy 794.6433715820312
	HOP 2 optimized energy -37.92913055419922


/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/dgl/core.py:82: DGLWarning: The input graph for the user-defined edge function does not contain valid edges
  dgl_warning(


	HOP 3 took 6.236618757247925 seconds
	HOP 3 previous energy -11.533863067626953
	HOP 3 optimized energy -37.70269012451172
	HOP 4 took 6.034566640853882 seconds
	HOP 4 previous energy 20.550800323486328
	HOP 4 optimized energy -39.86009216308594
Structure 0 Min energy -39.86009216308594
Structure 1
	HOP 0 took 6.015657663345337 seconds
	HOP 0 previous energy 3.8720130920410156
	HOP 0 optimized energy -32.57892608642578
	HOP 1 took 5.489787578582764 seconds
	HOP 1 previous energy -32.57892608642578
	HOP 1 optimized energy -32.6783447265625


/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/ase/constraints.py:2636: ComplexWarning: Casting complex values to real discards the imaginary part
  Y[0:3, 0:3] = cur_deform_grad_log
/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/ase/constraints.py:2637: ComplexWarning: Casting complex values to real discards the imaginary part
  Y[3:6, 3:6] = cur_deform_grad_log
/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/ase/constraints.py:2638: ComplexWarning: Casting complex values to real discards the imaginary part
  Y[0:3, 3:6] = - virial @ expm(-cur_deform_grad_log)
/global/homes/r/rithwiks/.conda/envs/matdeeplearn/lib/python3.9/site-packages/ase/constraints.py:2604: ComplexWarning: Casting complex values to real discards the imaginary part
  pos[natoms:] = logm(self.deform_grad())


	HOP 2 took 6.010356426239014 seconds
	HOP 2 previous energy -32.6783447265625
	HOP 2 optimized energy -32.737327575683594
	HOP 3 took 6.5199103355407715 seconds
	HOP 3 previous energy -21.42911148071289
	HOP 3 optimized energy -29.53144073486328
	HOP 4 took 6.1975462436676025 seconds
	HOP 4 previous energy 272.6516418457031
	HOP 4 optimized energy -37.3641471862793
Structure 1 Min energy -37.3641471862793
Structure 2
	HOP 0 took 5.115128040313721 seconds
	HOP 0 previous energy 532.0285034179688
	HOP 0 optimized energy -24.970951080322266
	HOP 1 took 0.008947372436523438 seconds
	HOP 1 previous energy -10.485706329345703
	HOP 1 optimized energy -10.485706329345703
	HOP 2 took 0.008466482162475586 seconds
	HOP 2 previous energy -10.485706329345703
	HOP 2 optimized energy -10.485706329345703
	HOP 3 took 0.009031534194946289 seconds
	HOP 3 previous energy -10.926040649414062
	HOP 3 optimized energy -10.926040649414062
	HOP 4 took 0.008983612060546875 seconds
	HOP 4 previous energy -8.7560

Compare best generated structures with true structure (if occurs)

In [66]:
# Comparisons require ASE atoms
minima_list_ase = dict_to_atoms(minima_list_ase)
minima_list_batch = dict_to_atoms(minima_list_batch)
minima_list_mace = dict_to_atoms(minima_list_mace)
minima_list_m3gnet = dict_to_atoms(minima_list_m3gnet)

In [69]:
adaptor = AseAtomsAdaptor
structure_matcher = StructureMatcher(ltol = 0.3, stol = 0.3, angle_tol = 5, primitive_cell = True, scale = True)
for i in range(len(minima_list_ase)):
    print('Structure', i)
    print('ase:', structure_matcher.fit(adaptor.get_structure(read_structure), adaptor.get_structure(minima_list_ase[i])))
    print('batch:', structure_matcher.fit(adaptor.get_structure(read_structure), adaptor.get_structure(minima_list_batch[i])))
    print('mace:', structure_matcher.fit(adaptor.get_structure(read_structure), adaptor.get_structure(minima_list_mace[i])))
    print('m3gnet:', structure_matcher.fit(adaptor.get_structure(read_structure), adaptor.get_structure(minima_list_m3gnet[i])))

Structure 0
ase: False
batch: False
mace: False
m3gnet: False
Structure 1
ase: False
batch: False
mace: False
m3gnet: False
Structure 2
ase: False
batch: False
mace: False
m3gnet: False
Structure 3
ase: False
batch: False
mace: False
m3gnet: False
Structure 4
ase: False
batch: False
mace: False
m3gnet: False


Quantify structure similairy, continous from 1 to 0
see: https://docs.materialsproject.org/methodology/materials-methodology/related-materials

In [68]:
ssf = SiteStatsFingerprint(CrystalNNFingerprint.from_preset('ops', distance_cutoffs=None, x_diff_weight=0), stats=('mean', 'std_dev', 'minimum', 'maximum'))
for i in range(len(minima_list_ase)):
    print('Structure', i)
    target = np.array(ssf.featurize(adaptor.get_structure(read_structure)))
    mdl = np.array(ssf.featurize(adaptor.get_structure(minima_list_ase[i])))
    mdl_batch = np.array(ssf.featurize(adaptor.get_structure(minima_list_batch[i])))
    mace = np.array(ssf.featurize(adaptor.get_structure(minima_list_mace[i])))
    m3gnet = np.array(ssf.featurize(adaptor.get_structure(minima_list_m3gnet[i])))
    print('Distance between target and mdl: {:.4f}'.format(np.linalg.norm(target - mdl)))
    print('Distance between target and mdl_batch: {:.4f}'.format(np.linalg.norm(target - mdl_batch)))
    print('Distance between target and mace: {:.4f}'.format(np.linalg.norm(target - mace)))
    print('Distance between target and m3gnet: {:.4f}'.format(np.linalg.norm(target - m3gnet)))

Structure 0
Distance between target and mdl: 1.8878
Distance between target and mdl_batch: 1.9492
Distance between target and mace: 2.1259
Distance between target and m3gnet: 2.0522
Structure 1
Distance between target and mdl: 2.0297
Distance between target and mdl_batch: 1.8226
Distance between target and mace: 2.1101
Distance between target and m3gnet: 2.0161
Structure 2
Distance between target and mdl: 2.3042
Distance between target and mdl_batch: 2.0938
Distance between target and mace: 2.3378
Distance between target and m3gnet: 2.5642
Structure 3
Distance between target and mdl: 2.2343
Distance between target and mdl_batch: 2.2104
Distance between target and mace: 2.3647
Distance between target and m3gnet: 2.3115
Structure 4
Distance between target and mdl: 2.3519
Distance between target and mdl_batch: 2.2714
Distance between target and mace: 2.0579
Distance between target and m3gnet: 2.1633
